In [14]:
import numpy as np
from itertools import product
import pandas as pd
import os, json, sys
import matplotlib.pyplot as plt


In [15]:
# cached interpolators
from functools import lru_cache
from scipy.interpolate import interp1d
import numpy as np


def interpolate_element_data(time_data, value_data, fit_type='linear'):
    """
    Interpolates the element data with an option for linear or logarithmic fitting.
    Returns a cached interpolation function.
    """
    time_data = np.array(time_data)
    value_data = np.array(value_data)

    if fit_type == 'log':
        min_positive = np.min(value_data[value_data > 0]) if np.any(value_data > 0) else 1e-10
        value_data = np.log(np.where(value_data > 0, value_data, min_positive))
        interpolator = interp1d(time_data, value_data, kind='linear', fill_value="extrapolate", bounds_error=False)
        # Cache the result of the exponentiated interpolation
        @lru_cache(maxsize=None)  # No limit to cache size
        def cached_interpolator(x):
            return np.exp(interpolator(x))
        return cached_interpolator
    else:
        interpolator = interp1d(time_data, value_data, kind='linear', fill_value="extrapolate")
        # Cache the linear interpolation
        @lru_cache(maxsize=None)  # No limit to cache size
        def cached_interpolator(x):
            return interpolator(x)
        return cached_interpolator

# get a list of the json files in the ../fispact_data directory

def create_interpolators(data_path, fit_type='linear', verbose=False):
    """
    Creates interpolation functions for each element based on data from JSON files.

    Parameters:
    data_path (str): The path to the directory containing the JSON files.

    Returns:
    dict: A dictionary where the keys are element names and the values are interpolation functions.

    """
    json_files = [f for f in os.listdir(data_path) if f.endswith('.json')]
    if verbose:
        print(json_files)

    # Load the data from the json files and create interpolation functions for each element
    element_data = {}
    for file in json_files:
        data = json.load(open(os.path.join('./fispact_data', file), 'r'))
        element = file.split('.')[0]
        time_data = [entry['cooling_time'] for entry in data['inventory_data']]
        value_data = [entry['dose_rate']['dose'] for entry in data['inventory_data']]
        element_data[element] = interpolate_element_data(time_data, value_data, fit_type=fit_type)
    
    return element_data
import pandas as pd
import plotly.express as px

import pandas as pd

def create_dataframe_from_dict(data_dict,optional_column_name = 'fitness'):
    """
    Converts a dictionary with nested 'composition' and optional 'fitness' into a DataFrame.
    
    Args:
    data_dict (dict): Dictionary with keys as indices and values containing 'composition' and optionally 'fitness'.
    
    Returns:
    pd.DataFrame: DataFrame with compositions and fitness as columns.
    """
    # Prepare lists to hold compositions and fitness values
    compositions = []
    fitness_values = []
    
    # Iterate through the dictionary to extract compositions and fitness values
    for key, value in data_dict.items():
        compositions.append(value['composition'])
        # Check if fitness is present and if it's a list or a scalar

        if optional_column_name in value and isinstance(value[optional_column_name], (int, float)):
            fitness_values.append(value[optional_column_name])  # Assuming fitness is always a list with one item
        elif optional_column_name in value and isinstance(value[optional_column_name], list):
            fitness_values.append(value[optional_column_name][0])

    # Create DataFrame from compositions
    df_compositions = pd.DataFrame(compositions)
    
    # Add fitness column if fitness values were extracted
    if fitness_values:
        df_compositions[optional_column_name] = fitness_values
    
    return df_compositions


import plotly.graph_objects as go
import pandas as pd

def create_parallel_coordinates_plot(df, fitness_col=None, in_percent=False, num_compositions=100):
    """
    Creates an interactive parallel coordinates plot from a DataFrame using Plotly Graph Objects,
    limiting the number of compositions displayed to `num_compositions`.

    Args:
    df (pd.DataFrame): DataFrame containing composition data with columns as elements.
    fitness_col (str, optional): Column name for the fitness values, used for coloring.
    in_percent (bool, optional): Whether to convert the composition values to percentages.
    num_compositions (int, optional): The number of compositions to display on the plot.

    Returns:
    go.Figure: The interactive plot.
    """
    if in_percent:
        df = df.copy()  # Create a copy to avoid modifying the original DataFrame
        for col in df.columns:
            if col != fitness_col:
                df[col] *= 100

    # Sort the DataFrame based on fitness column and take the top compositions
    if fitness_col and fitness_col in df.columns:
        df = df.sort_values(by=fitness_col).head(num_compositions)

    # Define labels with or without percentage units
    labels = {col: f"{col} (wt%)" if in_percent else col for col in df.columns if col != fitness_col}
    
    # Setting up dimensions for the parallel coordinates plot
    dimensions = []
    color = None
    for col in df.columns:
        if col != fitness_col:
            dimensions.append(
                go.parcats.Dimension(values=df[col], label=labels[col])
            )
        else:
            # Use the fitness column as a color dimension
            color = df[fitness_col]

    # Create the parallel coordinates plot
    fig = go.Figure(
        data=go.Parcats(
            dimensions=dimensions,
            line=dict(
                color=color,
                colorscale=px.colors.diverging.Tealrose,
                cmin=color.min(),
                cmax=color.max()
            ),
            hoveron='color', # Enable hover for color (dimension values)
            hoverinfo='all',
            labelfont=dict(size=12),
            tickfont=dict(size=10),
            arrangement='freeform'
        )
    )
    
    return fig


In [16]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px  # To use for color scales and other utilities

def create_parallel_coordinates_plot(df, fitness_col=None, in_percent=False, num_compositions=100):
    """
    Enhanced interactive parallel coordinates plot with better aesthetics and clear representation of fitness.

    Args:
    df (pd.DataFrame): DataFrame containing composition data.
    fitness_col (str, optional): Column name for the fitness values, used for coloring.
    in_percent (bool, optional): Convert composition values to percentages.
    num_compositions (int, optional): Number of compositions to display on the plot.

    Returns:
    go.Figure: The interactive plot with improved aesthetics.
    """
    if in_percent:
        df = df.copy()  # Avoid modifying the original DataFrame
        for col in df.columns:
            if col != fitness_col:
                df[col] *= 100  # Convert to percentage

    # Sorting and selecting top compositions based on fitness
    if fitness_col and fitness_col in df.columns:
        df = df.sort_values(by=fitness_col, ascending=True).head(num_compositions)  # Sort for better fitness scores

    # Setting labels for dimensions, considering percentage units
    labels = {col: f"{col} (%)" if in_percent else col for col in df.columns if col != fitness_col}

    # Setup dimensions for the plot
    dimensions = [
        go.parcats.Dimension(values=df[col], label=labels[col]) for col in df.columns if col != fitness_col
    ]

    # Determine color scale based on fitness
    color = df[fitness_col] if fitness_col in df.columns else None

    fig = go.Figure(
        data=go.Parcats(
            dimensions=dimensions,
            line=dict(
                color=color,
                colorscale='Viridis',  # Using a perceptually uniform colorscale
                cmin=color.min(),
                cmax=color.max(),
                showscale=True  # Show color scale to indicate fitness values
            ),
            hoveron='color',  # Hover effect on color dimension
            hoverinfo='all',
            labelfont=dict(size=12),
            tickfont=dict(size=10),
            arrangement='freeform'
        )
    )

    fig.update_layout(
        title="Parallel Coordinates Plot for Material Compositions",
        title_font_size=20,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )

    return fig


In [17]:
#elements = ['', 'B', 'C', 'D', 'E']  # Example elements
element_interpolators = create_interpolators('./fispact_data', fit_type='log')

elements = list(element_interpolators.keys())
print(elements)


['V', 'Ti', 'Zr', 'W', 'Cr']


In [18]:
import numpy as np
from itertools import product
from tqdm import tqdm  # Make sure tqdm is imported
from itertools import product
import numpy as np

def get_steps(range_tuple, step):
    return np.arange(range_tuple[0], range_tuple[1] + step, step)

def _generate_valid_combinations(ranges, tolerance=0.01):
    for combo in product(*ranges):
        if np.abs(sum(combo) - 1.0) <= tolerance:
            yield combo

def generate_valid_combinations(ranges, tolerance=0.01):
    # Calculate total possible combinations (for progress bar estimation)
    total_combinations = np.prod([len(r) for r in ranges])
    progress_bar = tqdm(total=total_combinations, desc="Generating valid combinations")

    # Iterate through all possible combinations
    for combo in product(*ranges):
        progress_bar.update(1)
        if np.abs(sum(combo) - 1.0) <= tolerance:
            yield combo
    
    progress_bar.close()

def find_time_to_threshold(element_interpolators, composition, dose_threshold, time_range):
    # Define time steps, e.g., from 0 to 1e8 seconds, with a given resolution
    for time in np.linspace(time_range[0], time_range[1], 1000):  # Adjust number of steps as needed
        total_dose = sum(element_interpolators[element](time) * composition[element] for element in composition)
        if total_dose < dose_threshold:
            return time
    return None  # Return None if the threshold is never met

def adaptive_grid_search(element_interpolators, comp_range, limits, step=0.001, tolerance=0.01, time_search=False, dose_threshold=1e-4, time_range=(0, 1e10)):
    elements = list(comp_range.keys())
    ranges = [get_steps(comp_range[element], step) for element in elements]

    # Use a generator to handle combinations
    valid_combinations = list(generate_valid_combinations(ranges, tolerance))
    print(f"Number of valid combinations: {len(valid_combinations)}")  # Debug statement

    compositions_data = {}  # Dictionary to store all compositions and their penalties
    best_score = float('inf')

    for idx, combo in tqdm(enumerate(valid_combinations), total=len(valid_combinations), desc='Evaluating combinations'):
        composition = dict(zip(elements, combo))

        if time_search:
            # Perform the time search for this composition
            time_to_threshold = find_time_to_threshold(element_interpolators, composition, dose_threshold, time_range)
            compositions_data[idx] = {'composition': composition, 'time_to_threshold': time_to_threshold}
        else:
            # Calculate penalties as before
            total_penalty = 0
            for limit in limits:
                time = limit['time']
                required_limit = limit['limit']
                importance = limit['importance']
                total_value = sum(element_interpolators[element](time) * composition[element] for element in elements)
                delta = max(0, total_value - required_limit)
                total_penalty += delta * importance

            compositions_data[idx] = {'composition': composition, 'fitness': [total_penalty]}
            if total_penalty < best_score:
                best_score = total_penalty

    return compositions_data

# Example limits
limits = [
    {'time': 3.41e7 / 12, 'limit': 1e4, 'importance': 0.5},
    {'time': 3.41e7 * 2, 'limit': 1e2 / 2, 'importance': 1},
    {'time': 3.41e7 * 10, 'limit': 1e0, 'importance': 2},
    {'time': 3.41e7 * 100, 'limit': 1e-4, 'importance': 4}
]



In [30]:
# Example usage, assuming element_interpolators and limits are defined
#comp_range = {'V': (0.75, 1.0), 'Cr': (0.01, 0.25), 'Ti': (0.01, 0.3), 'W': (0.01, 0.2), 'Zr': (0.01, 0.2)}
comp_range = {'V' : (0.33, 1.0), 'Cr' : (0.01, 0.33), 'Ti' : (0.01, 0.33)}

composition_data = adaptive_grid_search(element_interpolators, 
                                        comp_range, 
                                        limits,
                                        step=0.01)


Number of valid combinations: 2153


Evaluating combinations: 100%|██████████| 2153/2153 [00:00<00:00, 156061.39it/s]


In [33]:
# Example usage, assuming element_interpolators and limits are defined
#comp_range = {'V': (0.75, 1.0), 'Cr': (0.01, 0.25), 'Ti': (0.01, 0.3), 'W': (0.01, 0.2), 'Zr': (0.01, 0.2)}
#comp_range = {'V' : (0.33, 1.0), 'Cr' : (0.01, 0.33), 'Ti' : (0.01, 0.33)}
comp_range = {'V' : (0.01, 1.0), 'Cr' : (0.01, 1.0), 'Ti' : (0.01, 1.0)}


composition_data = adaptive_grid_search(element_interpolators, 
                                        comp_range, 
                                        limits,
                                        step=0.01,
                                        tolerance=0.01,
                                        time_search=True,
                                        dose_threshold=1e-2,
                                        time_range=(1e0, 1e10))


Generating valid combinations: 100%|██████████| 1000000/1000000 [00:00<00:00, 1244745.75it/s]


Number of valid combinations: 5705


Evaluating combinations: 100%|██████████| 5705/5705 [00:00<00:00, 39103.66it/s]


In [34]:
optional_column_name = 'time_to_threshold'   # Optional column name for the time to threshold data change to fitness
# Convert the resulting dictionary to a DataFrame
df_compositions = create_dataframe_from_dict(composition_data,optional_column_name=optional_column_name)
#print(df_compositions.head())

# save the dataframe to a json for future use
df_compositions.to_json('rough_df_compositions_time_data.json')

# let's create a smaller version of the dataframe, selecting 10,000 compositions at random
#test_df_compositions = df_compositions.sample(n=10000, random_state=42)
#test_df_compositions.to_json('test_df_compositions_time_data.json')

# select the df compositions that have a value for column 'V' greater than 0.7
#df_compositions = df_compositions[df_compositions['V'] > 0.7]

# randomly select 1000 compositions from df_compositions 
#rand_df_compositions = df_compositions.sample(n=1000, random_state=42)

# round the values in the df_compositions to 4 decimal places and sort the values in the V column by highest to lowest
#rand_df_compositions = rand_df_compositions.sort_values(by='V', ascending=False)
#rand_df_compositions = rand_df_compositions.round(4)
# convert the time_to_threshold values to years and round to 2 decimal places
#rand_df_compositions['time_to_threshold'] = round(rand_df_compositions['time_to_threshold'] / (60*60*24*365),2)

#print(rand_df_compositions.head())

# Optionally create a plot
#fig = create_parallel_coordinates_plot(rand_df_compositions, fitness_col=optional_column_name, in_percent=True)
#fig.show()

In [44]:
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return pd.DataFrame(data)

def _classify_compositions(df, limit):
    # Define categories based on time_to_threshold
    conditions = [
        (df['time_to_threshold'] <= limit),  # Within limit
        (df['time_to_threshold'] <= 10 * limit),  # Within an order of magnitude
        (df['time_to_threshold'] > 10 * limit)  # More than an order of magnitude
    ]
    colors = ['green', 'yellow', 'red']  # Corresponding colors
    df['color'] = np.select(conditions, colors, default='red')
    return df

def classify_compositions(df, limit, threshold=True):
    if threshold:
        # Define categories based on time_to_threshold
        conditions = [
            (df['time_to_threshold'] <= limit),  # Within limit
            (df['time_to_threshold'] <= 10 * limit),  # Within an order of magnitude
            (df['time_to_threshold'] > 10 * limit)  # More than an order of magnitude
        ]
        colors = ['green', 'yellow', 'red']  # Corresponding colors
        df['color'] = np.select(conditions, colors, default='red')
    else:
        # Create a heatmap based on the values of time_to_threshold
        df['color'] = np.log10(df['time_to_threshold'])  # Convert time to a logarithmic scale for better color differentiation
        max_log = df['color'].max()
        min_log = df['color'].min()
        df['color'] = (df['color'] - min_log) / (max_log - min_log)  # Normalize to 0-1 for color scale

        # Assign colors from red (high) to green (low) using a continuous color scale
        df['color'] = df['color'].apply(lambda x: px.colors.diverging.RdYlGn[abs(int(x*10)) % len(px.colors.diverging.RdYlGn)])
    
    return df

def merge_classifications(df1, df2, limit1, limit2):
    # Assume df1 and df2 are DataFrames with 'time_to_threshold' and same 'V', 'Cr', 'Ti' proportions
    df1 = classify_compositions(df1, limit1).copy()
    df2 = classify_compositions(df2, limit2).copy()

    # Merge based on composition keys assuming the compositions are identical in structure
    merged_df = pd.merge(df1, df2, on=['V', 'Cr', 'Ti'], suffixes=('_1', '_2'))

    # Define merged color logic
    conditions = [
        (merged_df['color_1'] == 'green') & (merged_df['color_2'] == 'green'),  # Both are green -> dark green
        ((merged_df['color_1'] == 'green') & (merged_df['color_2'] == 'yellow')) | ((merged_df['color_1'] == 'yellow') & (merged_df['color_2'] == 'green')),  # One green, one yellow -> light green
        (merged_df['color_1'] == 'yellow') & (merged_df['color_2'] == 'yellow'),  # Both are yellow -> yellow
        ((merged_df['color_1'] == 'yellow') & (merged_df['color_2'] == 'red')) | ((merged_df['color_1'] == 'red') & (merged_df['color_2'] == 'yellow')),  # One yellow, one red -> orange
        (merged_df['color_1'] == 'red') & (merged_df['color_2'] == 'red')  # Both are red -> red
    ]
    final_colors = ['darkgreen', 'lightgreen', 'yellow', 'orange', 'red']
    merged_df['color'] = np.select(conditions, final_colors, default='red')

    return merged_df

def plot_ternary(df, threshold=True, min_log = None, max_log = None):
    if threshold:
        # Use the categorical 'color' column from df
        marker_colors = df['color']
        showscale = False
    else:
        # Compute the log10 values of time_to_threshold
        log_time_to_threshold = np.log10(df['time_to_threshold'])
        if min_log is None:
            min_log = np.floor(log_time_to_threshold.min())
        if max_log is None:
            max_log = np.ceil(log_time_to_threshold.max())
        #min_log = np.floor(log_time_to_threshold.min())  # Floor the min value
        #max_log = np.ceil(log_time_to_threshold.max())   # Ceil the max value

        # Set the marker colors directly to log values for continuous scale
        marker_colors = log_time_to_threshold
        showscale = True  # Enable color scale for heatmap

    # Create ternary plot with appropriate settings
    fig = go.Figure(go.Scatterternary({
        'mode': 'markers',
        'a': df['V'],
        'b': df['Cr'],
        'c': df['Ti'],
        'marker': {
            'symbol': 100,
            'color': marker_colors,
            'cmin': min_log if not threshold else None,
            'cmax': max_log if not threshold else None,
            'colorscale': 'RdYlGn_r',  # Red to green reversed scale
            'colorbar': {'title': 'Log10(Time to Threshold)'} if showscale else None,
            'size': 5,
            'line': {'width': 2}
        }
    }))

    fig.update_layout({
        'ternary': {
            'sum': 1,
            'aaxis': {'title': 'V', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'baxis': {'title': 'Cr', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'caxis': {'title': 'Ti', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'}
        },
        #'title': 'Ternary Plot of Compositions'
    })
    
    return fig





In [57]:
# Use the functions
df = load_data('rough_df_compositions_time_data.json')

# convert time_to_threshold values to years
df['time_to_threshold'] = round(df['time_to_threshold'] / (60*60*24*365),2)
# sort by time_to_threshold, lowest first
df = df.sort_values(by='time_to_threshold', ascending=True)
print(df.head())

time_to_threshold_limit = 100 * 60 * 60 * 24 * 365  # Example limit
df_classified = classify_compositions(df, time_to_threshold_limit, threshold=False)
#fig = plot_ternary(df_classified, threshold=False,min_log=0.75, max_log=1)
fig = plot_ternary(df_classified, threshold=True)
fig.show()

# i want to also save the figure as a png
#fig.write_image('ternary_plot.png')


         V    Cr    Ti  time_to_threshold
5704  0.98  0.01  0.01               6.35
5580  0.85  0.03  0.12               6.35
5592  0.86  0.01  0.13               6.35
5593  0.86  0.02  0.12               6.35
5594  0.86  0.03  0.11               6.35


In [45]:
# load df from json file
df1 = load_data('rough_df_compositions_time_data.json')
df2 = load_data('rough_df_compositions_time_data_1e-4.json')
merged_df = merge_classifications(df1, df2, 7.5 * 60 * 60 * 24 * 365, 100 * 60 * 60 * 24 * 365)
fig = plot_ternary(merged_df, threshold=True)
fig.write_image('threshold_ternary_plot_merged.png')

In [47]:
# create a list of dictionaries from merged_df, each dictionary will be {'V' : value, 'Cr' : value, 'Ti' : value, 'color' : value}
data = []
for index, row in merged_df.iterrows():
    data.append({'V' : row['V'], 'Cr' : row['Cr'], 'Ti' : row['Ti'], 'color' : row['color']})

# save the dicitonary to a json file
with open('ternary_plot_data.json', 'w') as file:
    json.dump(data, file)

In [55]:
data = json.load(open('ternary_plot_data.json','r'))

print(data[0])

for d in data:
    print(d['color'])

list_of_x_cr = []
list_of_x_ti = []

# go through each dictionary in the data and if the 'color' key is green, append the value in 'Cr' to list_of_x_cr and the value in 'Ti' to list_of_x_ti
for d in data:
    if d['color'] == 'darkgreen':
        list_of_x_cr.append(d['Cr'])
        list_of_x_ti.append(d['Ti'])

{'V': 0.01, 'Cr': 0.01, 'Ti': 0.98, 'color': 'yellow'}
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
yellow
yellow
yellow
yellow
yellow
yellow
yellow
ye

In [56]:
print(list_of_x_cr)

[0.29, 0.28, 0.28, 0.29, 0.29, 0.3, 0.27, 0.27, 0.28, 0.28, 0.29, 0.29, 0.3, 0.3, 0.31, 0.26, 0.26, 0.27, 0.27, 0.28, 0.28, 0.29, 0.29, 0.3, 0.3, 0.31, 0.32, 0.25, 0.25, 0.26, 0.26, 0.27, 0.27, 0.28, 0.28, 0.29, 0.29, 0.3, 0.3, 0.31, 0.31, 0.32, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.29, 0.3, 0.3, 0.31, 0.31, 0.32, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.29, 0.3, 0.3, 0.31, 0.32, 0.33, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.29, 0.3, 0.31, 0.32, 0.33, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.2, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.19, 0.19, 0.2, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.18, 0.18, 0.19, 0.19, 0.2, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.17, 0.17, 0.18, 0.18, 0.19, 0.19, 0.2, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32